When agents take actions in the real world, we need robust safety mechanisms. Let’s explore patterns for building safe action systems using a calendar coordination example.

#Pattern 1: Reversible Actions
When agents interact with the real world, mistakes can happen. Even if an agent makes the right decision with the information it has, external circumstances might change that make an action undesirable after it’s taken. Reversible actions provide a safety net, allowing us to “undo” operations when things go wrong.

The key to making actions reversible is capturing enough information during the execution to fully reverse the operation later. This isn’t always straightforward - for example, deleting a calendar event is easy to reverse if you saved all the event details, but reversing an email that’s already been read requires careful thought about what “reversal” means in your system. Sometimes, the best we can do is send a follow-up correction.

Let’s examine how we might structure actions so they can be undone:

In [ ]:
class ReversibleAction:
    def __init__(self, execute_func, reverse_func):
        self.execute = execute_func
        self.reverse = reverse_func
        self.execution_record = None

    def run(self, **args):
        """Execute action and record how to reverse it."""
        result = self.execute(**args)
        self.execution_record = {
            "args": args,
            "result": result,
            "timestamp": datetime.now().isoformat()
        }
        return result

    def undo(self):
        """Reverse the action using recorded information."""
        if not self.execution_record:
            raise ValueError("No action to reverse")
        return self.reverse(**self.execution_record)

# Example using reversible actions
create_event = ReversibleAction(
    execute_func=calendar.create_event,
    reverse_func=lambda **record: calendar.delete_event(record["result"]["event_id"])
)

send_invite = ReversibleAction(
    execute_func=calendar.send_invite,
    reverse_func=lambda **record: calendar.cancel_invite(record["result"]["invite_id"])
)

Now, if we record a log of the agent’s actions, we can undo them if needed. The agent itself can also be given a “revert” tool to undo its most recent action or revert all actions under certain circumstances.

#Pattern 2: Transaction Management
Individual reversible actions are helpful, but real tasks often require multiple coordinated actions. For example, scheduling a meeting might involve checking availability, creating a calendar event, and sending notifications. Transaction management lets us treat these related actions as a single unit that either completely succeeds or completely fails.

The transaction pattern becomes particularly powerful when combined with reversible actions. If any step in a multi-step process fails, we can automatically roll back all previous steps. This prevents inconsistent states where, for example, a calendar event exists but notifications weren’t sent, or notifications were sent but the event wasn’t created. It’s like having an “undo” button for entire sequences of operations.

Here is a simple transaction management system that we might use to collect the actions that an agent is executing during the agent loop and then either commit or rollback all of them at the end of the loop.

In [ ]:
class ActionTransaction:
    def __init__(self):
        self.actions = []
        self.executed = []
        self.committed = False
        self.transaction_id = str(uuid.uuid4())

    def add(self, action: ReversibleAction, **args):
        """Queue an action for execution."""
        if self.committed:
            raise ValueError("Transaction already committed")
        self.actions.append((action, args))

    async def execute(self):
        """Execute all actions in the transaction."""
        try:
            for action, args in self.actions:
                result = action.run(**args)
                self.executed.append(action)
        except Exception as e:
            # If any action fails, reverse everything done so far
            await self.rollback()
            raise e

    async def rollback(self):
        """Reverse all executed actions in reverse order."""
        for action in reversed(self.executed):
            await action.undo()
        self.executed = []

    def commit(self):
        """Mark transaction as committed."""
        self.committed = True

#Pattern 3: Staged Execution with Review
Sometimes we want more control than just automatic rollback on failure. Staged execution allows us to plan out a series of actions, review them for safety and correctness, and only then execute them. This is particularly valuable when some actions are more reversible than others, or when we want to give a human or more capable AI system a chance to review plans before execution.

The review phase can catch problems that might not be apparent when looking at individual actions. For example, while each step in setting up a large meeting might be valid on its own, a reviewer might notice that the combination of actions would overwhelm a team’s schedule or violate company policies about meeting sizes. This pattern also allows for optimization - a reviewer might notice more efficient ways to accomplish the same goal.

Now let’s create a system that stages changes for review before execution:

In [ ]:
class StagedActionEnvironment(Environment):
    def __init__(self):
        self.staged_transactions = {}
        self.llm = None  # High-capability LLM for review

    def stage_actions(self, task_id: str) -> ActionTransaction:
        """Create a new transaction for staging actions."""
        transaction = ActionTransaction()
        self.staged_transactions[task_id] = transaction
        return transaction

    def review_transaction(self, task_id: str) -> bool:
        """Have LLM review staged actions for safety."""
        transaction = self.staged_transactions.get(task_id)
        if not transaction:
            raise ValueError(f"No transaction found for task {task_id}")

        # Create a description of staged actions
        staged_actions = [
            f"Action: {action.__class__.__name__}\nArgs: {args}"
            for action, args in transaction.actions
        ]

        # The safest way to do this would be to send it for human review, but we can also imagine having a more capable AI system review it before the human to minimize the number of reviews that the human has to do. The more capable AI can review and reject potentially problematic actions earlier.

        review_prompt = f"""Review these staged actions for safety:

        Task ID: {task_id}

        Staged Actions:
        {staged_actions}

        Consider:
        1. Are all actions necessary for the task?
        2. Could any action have unintended consequences?
        3. Are the actions in a safe order?
        4. Is there a safer way to achieve the same goal?

        Should these actions be approved?
        """

        response = self.llm.generate(review_prompt)

        # If approved, notify the human and ask if
        # they want to proceed
        return "approved" in response.lower()

# Example usage:
async def schedule_team_meeting(env: StagedActionEnvironment,
                              attendees: List[str],
                              duration: int):
    """Schedule a team meeting with safety checks."""
    task_id = str(uuid.uuid4())
    transaction = env.stage_actions(task_id)

    # Check availability (execute immediately)
    available_slots = calendar.check_availability(attendees, duration)
    if not available_slots:
        return {"error": "No available time slots"}

    best_slot = available_slots[0]

    # Stage the event creation
    transaction.add(create_event,
                   title="Team Meeting",
                   time=best_slot,
                   duration=duration)

    # Draft email (execute immediately)
    email_draft = email.draft_message(
        to=attendees,
        subject="Team Meeting",
        body=f"Team meeting scheduled for {best_slot}"
    )

    # Stage the email send
    transaction.add(send_email,
                   draft_id=email_draft.id)

    # Review staged actions...send to human review
    # or more capable AI for initial filtering
    if env.review_transaction(task_id):
        await transaction.execute()
        transaction.commit()
        return {"status": "scheduled"}
    else:
        return {"status": "rejected"}

#Pattern 4: Single Safe Tool vs Multiple Risky Tools
At first glance, having many small, focused tools seems like good design - it follows the Unix philosophy of “do one thing and do it well.” However, when it comes to agent safety, this approach can backfire. Small, focused tools often lack the context to make safe decisions, and they put the burden of correct orchestration on the agent.

A single comprehensive tool, in contrast, can encapsulate not just the mechanics of an operation but also the business rules and safety constraints that make it correct. Instead of hoping the agent will check availability before scheduling a meeting, verify attendees before sending invites, and handle timezone conversions correctly, we can build all of these safety checks into a single tool. This makes it much harder for the agent to accidentally create problematic situations, even if it tries to use the tool in unexpected ways.

Let’s compare two approaches to meeting scheduling:

In [ ]:
# Approach 1: Multiple loosely constrained tools
@register_tool(description="Create a calendar event")
def create_calendar_event(action_context: ActionContext,
                         title: str,
                         time: str,
                         attendees: List[str]) -> dict:
    """Create a calendar event."""
    return calendar.create_event(title=title,
                               time=time,
                               attendees=attendees)

@register_tool(description="Send email to attendees")
def send_email(action_context: ActionContext,
               to: List[str],
               subject: str,
               body: str) -> dict:
    """Send an email."""
    return email.send(to=to, subject=subject, body=body)

@register_tool(description="Update calendar event")
def update_event(action_context: ActionContext,
                 event_id: str,
                 updates: dict) -> dict:
    """Update any aspect of a calendar event."""
    return calendar.update_event(event_id, updates)










# Approach 2: Single comprehensive safe tool
@register_tool(description="Schedule a team meeting safely")
def schedule_team_meeting(action_context: ActionContext,
                         title: str,
                         description: str,
                         attendees: List[str],
                         duration_minutes: int,
                         timeframe: str = "next_week") -> dict:
    """
    Safely schedule a team meeting with all necessary coordination.

    This tool:
    1. Verifies all attendees are valid
    2. Checks calendar availability
    3. Creates the event at the best available time
    4. Sends appropriate notifications
    5. Handles all error cases
    """
    # Input validation
    if not 15 <= duration_minutes <= 120:
        raise ValueError("Meeting duration must be between 15 and 120 minutes")

    if len(attendees) > 10:
        raise ValueError("Cannot schedule meetings with more than 10 attendees")

    # Verify attendees
    valid_attendees = validate_attendees(attendees)
    if len(valid_attendees) != len(attendees):
        raise ValueError("Some attendees are invalid")

    # Find available times
    available_slots = find_available_times(
        attendees=valid_attendees,
        duration=duration_minutes,
        timeframe=timeframe
    )

    if not available_slots:
        return {
            "status": "no_availability",
            "message": "No suitable time slots found"
        }

    # Create event at best time
    event = calendar.create_event(
        title=title,
        description=description,
        time=available_slots[0],
        duration=duration_minutes,
        attendees=valid_attendees
    )

    # Send notifications
    notifications.send_meeting_scheduled(
        event_id=event.id,
        attendees=valid_attendees
    )

    return {
        "status": "scheduled",
        "event_id": event.id,
        "scheduled_time": available_slots[0]
    }

The single comprehensive tool is safer because:

It enforces proper sequencing of operations
It includes all necessary validation
It handles error cases consistently
It prevents misuse of individual operations
With the multiple tool approach, an agent might:

Create an event without checking availability
Send emails without creating the event
Update events without notifying attendees
Exceed reasonable size limits
The single tool encapsulates the business logic of what makes a “safe” meeting scheduling operation, making it much harder for the agent to create problematic situations.